In [4]:
import tensorflow_addons as tfa

import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils

C:\Users\Ambrose\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [5]:
#load data and tag

reviews = []
responses = []

with open('review.dat', 'rb') as f:
    rev = pickle.load(f)
with open('reply.dat', 'rb') as f:
    rep = pickle.load(f)
    
for i in range(len(rep)):
    responses.append( '<START> ' + rep[i] + ' <END>' )
    
reviews = rev

In [6]:
reviews = reviews[0:int(len(reviews)/2)]
responses = responses[0:int(len(responses)/2)]

In [7]:
#tokenize
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( reviews + responses )

VOCAB_SIZE = len( tokenizer.word_index )+1
#print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

In [8]:
# encoder_input_data
tokenized_reviews = tokenizer.texts_to_sequences(reviews)
maxlen_reviews = max([len(x) for x in tokenized_reviews])
padded_reviews = preprocessing.sequence.pad_sequences(tokenized_reviews , maxlen=maxlen_reviews , padding='post')
encoder_input_data = np.array(padded_reviews)
#print(encoder_input_data.shape , maxlen_reviews)

# decoder_input_data
tokenized_responses = tokenizer.texts_to_sequences(responses)
maxlen_responses = max([len(x) for x in tokenized_responses])
padded_responses = preprocessing.sequence.pad_sequences(tokenized_responses , maxlen=maxlen_responses , padding='post')
decoder_input_data = np.array(padded_responses)
#print(decoder_input_data.shape , maxlen_responses)

# decoder_output_data
tokenized_responses = tokenizer.texts_to_sequences(responses)
for i in range(len(tokenized_responses)) :
    tokenized_responses[i] = tokenized_responses[i][1:]
padded_responses = preprocessing.sequence.pad_sequences(tokenized_responses , maxlen=maxlen_responses , padding='post')
onehot_responses = utils.to_categorical(padded_responses , VOCAB_SIZE)
decoder_output_data = np.array(onehot_responses)
#print( decoder_output_data.shape )

(536, 961) 961
(536, 809) 809
(536, 809, 8656)


In [9]:
#model creation

encoder_inputs = tf.keras.layers.Input(shape=(maxlen_reviews , ))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)( encoder_embedding )
encoder_states = [state_h , state_c]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_responses ,  ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 961)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 809)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 961, 200)     1731200     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 809, 200)     1731200     input_2[0][0]                    
______________________________________________________________________________________________

In [10]:
#attempt larger batch size if comp can handle it (50)
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=25, epochs=150 ) 

# this method of saving should not be trusted
# reloading this model breaks system... saving weights better
model.save('model.h5') 

Epoch 1/150
22/22 [==============================] - 19s 381ms/step - loss: 0.7917
Epoch 2/150
22/22 [==============================] - 9s 394ms/step - loss: 0.7084
Epoch 3/150
22/22 [==============================] - 9s 389ms/step - loss: 0.6996
Epoch 4/150
22/22 [==============================] - 9s 394ms/step - loss: 0.6901
Epoch 5/150
22/22 [==============================] - 8s 380ms/step - loss: 0.6824
Epoch 6/150
22/22 [==============================] - 9s 390ms/step - loss: 0.6759
Epoch 7/150
22/22 [==============================] - 9s 394ms/step - loss: 0.6690
Epoch 8/150
22/22 [==============================] - 9s 402ms/step - loss: 0.6612
Epoch 9/150
22/22 [==============================] - 9s 407ms/step - loss: 0.6548
Epoch 10/150
22/22 [==============================] - 9s 401ms/step - loss: 0.6480
Epoch 11/150
22/22 [==============================] - 9s 413ms/step - loss: 0.6418
Epoch 12/150
22/22 [==============================] - 9s 407ms/step - loss: 0.6398
Epoch 13/150

22/22 [==============================] - 9s 398ms/step - loss: 0.2267
Epoch 100/150
22/22 [==============================] - 9s 403ms/step - loss: 0.2241
Epoch 101/150
22/22 [==============================] - 9s 404ms/step - loss: 0.2206
Epoch 102/150
22/22 [==============================] - 9s 396ms/step - loss: 0.2187
Epoch 103/150
22/22 [==============================] - 9s 396ms/step - loss: 0.2154
Epoch 104/150
22/22 [==============================] - 9s 401ms/step - loss: 0.2131
Epoch 105/150
22/22 [==============================] - 9s 404ms/step - loss: 0.2107
Epoch 106/150
22/22 [==============================] - 9s 402ms/step - loss: 0.2081
Epoch 107/150
22/22 [==============================] - 9s 406ms/step - loss: 0.2050
Epoch 108/150
22/22 [==============================] - 10s 441ms/step - loss: 0.2024
Epoch 109/150
22/22 [==============================] - 10s 457ms/step - loss: 0.2007
Epoch 110/150
22/22 [==============================] - 9s 415ms/step - loss: 0.1969
Epoc

In [56]:
# input review to encoder
# input hidden and mem states, and replies to decoder
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(200,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [57]:
#convert input str to tokens
def str_to_tokens(sentence : str):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        try:
            tokens_list.append(tokenizer.word_index[word]) 
        except:
            pass
    return preprocessing.sequence.pad_sequences([tokens_list],maxlen=maxlen_reviews,padding='post')

In [58]:
# this does not work. 

'''
# reloading model because I shut down my computer

import os

model = tf.keras.models.load_model('model.h5')
model.summary()
'''

"\n# reloading model because I shut down my computer\n\nimport os\n\nmodel = tf.keras.models.load_model('model.h5')\nmodel.summary()\n"

In [59]:
# import eval metrics and get test set

import nltk.translate.bleu_score as bleu
import nltk.translate.nist_score as nist

test_reviews = rev[int(len(rev)/2):]
test_responses = rep[int(len(rep)/2):]

In [62]:
# cycle through test set and compile lists for scores and 

enc_model , dec_model = make_inference_models()

nScore = []
bScore = []
translation = []

for i in range(100):
    states_values = enc_model.predict(str_to_tokens(test_reviews[i]))
    empty_target_seq = np.zeros((1,1))
    empty_target_seq[0,0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format(word)
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_responses:
            stop_condition = True
            
        empty_target_seq = np.zeros((1,1))  
        empty_target_seq[0,0] = sampled_word_index
        states_values = [h,c] 
    
    translation.append(decoded_translation[1:-4])
    bScore.append(bleu.sentence_bleu(test_responses[i][8:-4], decoded_translation[1:-4]))
    nScore.append(nist.sentence_nist(test_responses[i][8:-4], decoded_translation[1:-4]))


In [63]:
# print min max, and avgs
print("n average:", sum(nScore) / len(nScore))
print("n min:", min(nScore))
print("n max:", max(nScore))
print()
print("b average:", sum(bScore) / len(bScore))
print("b min:", min(bScore))
print("b max:", max(bScore))

## for ref, the hw model returned the following scores on the same dataset
# avg: 0.000920
# min: 0.000000
# max: 0.091984

n average: 0.05435444922513263
n min: 0.00437513244752933
n max: 0.34968416183086115

b average: 1.0868379274593338e-231
b min: 6.24411123256442e-232
b max: 1.7296736363974997e-231


In [64]:
# get idx of min and max 
max_value = max(nScore)
min_value = min(nScore)
inmin = nScore.index(min_value)
inmax = nScore.index(max_value)

In [65]:
# look for average, below avg, etc.
j = 0
for i in nScore:
    if 0.10 < i:
        print(j,":",i)
    j+=1

42 : 0.18440511541192503
46 : 0.10432861097730246
58 : 0.10251566939396085
73 : 0.10362749302278992
83 : 0.34968416183086115
95 : 0.12515464018114972


In [66]:
# below average reply
print(test_reviews[19])
print()
print(test_responses[19])
print()
print(translation[19])

Now I know why this place has bad reviews!Very unprofessional servers and the manager Kevin is an idiot.

Andy - we'd like you to send us some more details about what happened during your visit so we can try to help. Please email Social@LongHornSteakhous… with your full name, receipt or order number, and phone number. Remind us you visited our Matthews location and that you originally contacted us on Yelp.

we’re so sorry to hear about your experience you had at us at our center please let us know if you need any questions or concerns please feel free to let us know we can do more than exceptional we have been a solution to help you in any concerns and you can email us and thank you for reaching out of your day gratefully christian brothers coddle creek


In [67]:
#average reply
print(test_reviews[1])
print()
print(test_responses[1])
print()
print(translation[1])

Market 168 is a good alternative to the busier Asian markets in the surrounding area. Staffing has always been helpful and accommodating.Pricing has been adjusted to be competitive with the other markets in town. Hot deli offers items not found at other market steam tables.

Dear Sherpa O.,Thank you for taking the time to write this review. We appreciate your patronage, and hope to see you again soon!Sincerely,168 Market Customer Service Team

thanks for your feedback and we really appreciate you taking the time to share your experience with us we appreciate your business


In [68]:
#above average reply
print(test_reviews[58])
print()
print(test_responses[58])
print()
print(translation[58])

First to ReviewMy husband is military and the base we are stationed at recommended this place for printing. THEY ARE VETERAN OWNED AND OPERATED. Two nice guys work there and are very helpful. The prices are amazing, the place is clean and very organized. I will definitely be returning. So much closer and faster than the post officeOutside of We Got MailThe outside of We Got Mail

thank you so much for your review. please come back in and take 15% off your total purchase. this is a thank you offer, and just remind me that i put this on: get any of the regular or brass keys made at .99 cents each (no limit!)!!!! just let us know that michael gave this offer!merry christmas!your friends from "we got mail"remeber all military get a 10% off thier total purchase at all times, and i will extend the .99 cent key offer to any military friends or family you refer!thanks again!

john we are so glad you had the great experience with you we hope to see you again soon


In [69]:
#max reply
print(test_reviews[inmax])
print()
print(test_responses[inmax])
print()
print(translation[inmax])

a lot people hate on panda express because it's not real chinese food. that's true. but that's not why i'm complaining. they didn't give me enough shrimp when i ordered the walnut shrimp and they charged me an extra 1.50!i feel like you'll get much better and more food if you to a buffet like pacific seafood buffet.

Hi John, thanks for bringing this to my attention. I'll check in with the restaurant team about this concern. Let me make it up to you. Please check your DM.

thank you jerica


In [70]:
#min reply
print(test_reviews[inmin])
print()
print(test_responses[inmin])
print()
print(translation[inmin])

Las Vegas, NV11052719303/17/2021Rain is everything you want in a business. Friendly service, fast scheduling and very reasonable prices. Our water softener stopped working and I called a few places to come check it. Rain was the only company that would come out without a service charge. As expected our valve was cracked and the unit was not fixable. They gave me a quote for a replacement on a new system which was very fair. We decided to wait to replace it. I decided to have it removed and called a couple plumbers who quoted me very high prices to remove it and cap the water line. I called Rain again and they were happy to come out and do it for me. They gave me a very reasonable price and came out the next morning. The job was done quickly and the old unit removed. I couldn't be happier with the fast friendly service and pricing. When we decide to finally replace it I will definitely be calling Rain!

Rain is everything you want in a business. Friendly service, fast scheduling and ver